# Import basic libraries

Let's import some basic stuff

### Fix for `fiona`

*`fiona` has an issue with GDAL 3.0. Better set your `GDAL_DATA` path to fiona's installation (contains GDAL 2.4.4) prior to running this script.*

In [ ]:
import geopandas
import pandas as pd
import osmnx as ox

# Load the OSM data

This will load the osm file to a network

In [ ]:
network_graph = ox.graph_from_file('../data/Breda/Breda extract.osm', simplify=False)
edges = ox.graph_to_gdfs(network_graph, nodes=False, node_geometry=False)

edges = edges.to_crs("EPSG:28992")

In [ ]:
len(edges)

Filter roads to only motorways:

In [ ]:
highway_types = [
    "primary",
    "secondary",
    "motorway",
    "trunk",
    "tertiary",
    "unclassified",
    "residential",

    "motorway_link",
    "trunk_link",
    "primary_link",
    "secondary_link",
    "tertiary_link",

    "living_street",
    "service",
    "pedestrian",
    "track",
    "bus_guideway",
    "escape",
    "raceway",
    "road"
]

road_edges = edges[(edges['highway'].isin(highway_types)) & (edges['area'] != 'yes')]

road_edges.to_file('output/road_edges.geojson', driver='GeoJSON')
len(road_edges)

# Load the BGT data

In [ ]:
bgt_roads = geopandas.read_file('../data/Breda/bgt_roads.geojson')

len(bgt_roads)

## Compute primary-secondary road polygons

We need to assign secondary road polygons (parking spaces, sidewalks) to the primary roads

### Filter primary roads

Primary types are those that vehicles are supposed to move:

In [ ]:
bgt_road_types = [
    'rijbaan lokale weg',
    'rijbaan regionale weg',
    # 'overweg' # Maybe?
]

bgt_roads['parent_gml_id'] = bgt_roads['gml_id']

bgt_main_roads = bgt_roads[bgt_roads['function'].isin(bgt_road_types)]

len(bgt_main_roads)

### Apply spatial join

*BROKEN: Now a primary road will get another primary road as parent!*

Find the secondary roads that touch the main ones:

In [ ]:
bgt_secondary_roads = bgt_roads[~bgt_roads['function'].isin(bgt_road_types)]

len(bgt_secondary_roads)

In [ ]:
joined = geopandas.sjoin(bgt_secondary_roads, bgt_main_roads, how='left', op='intersects')

joined = joined[['gml_id_left', 'function_left', 'gml_id_right', 'geometry']]
joined.columns = ['gml_id', 'function', 'parent_gml_id', 'geometry']

bgt_roads_parented = geopandas.GeoDataFrame(joined.groupby('gml_id').aggregate('first').reset_index(), crs='EPSG:28992')

len(bgt_roads_parented)

In [ ]:
bgt_roads_parented.columns

Concatenate everything:

In [ ]:
bgt_all_roads = pd.concat([bgt_roads_parented, bgt_main_roads[['gml_id', 'function', 'parent_gml_id', 'geometry']]])

len(bgt_all_roads)

bgt_all_roads.to_file('output/bgt_all_roads', driver='GeoJSON')

# Intersect roads network with polygon
First, we intersect the road lines with the BGT polygons (to create nodes at the polygon boundaries):

In [ ]:
roads = geopandas.overlay(road_edges, bgt_all_roads, how='intersection')

Then, dissolve the roads by `parent_gml_id` (just to group lines per BGT primary road polygon):

In [ ]:
roads_dissolved = roads.dissolve(by='parent_gml_id')

Export the intersected roads to `GeoJSON`:

In [ ]:
roads.to_file('output/roads.json', driver='GeoJSON')
roads_dissolved.to_file('output/roads_dissolved.geojson', driver='GeoJSON')

# Export to CityJSON

Let's export everything to CityJSON.

First, we'll define how the established intersected lines will be translated to `Road` objects:

In [ ]:
def process_line(line, vertices):
    points = [[x, y, 0] for x, y in list(line.coords)]
    indices = [i + len(vertices) for i in range(len(points))]
    for p in points:
        vertices.append(p)

    return indices

def process_geometry(geom, vertices):
    if geom.type == "LineString":
        indices = [process_line(geom, vertices)]
    else:
        indices = []
        
        for l in geom.geoms:
            indices.append(process_line(l, vertices))
        
    return indices

def create_geometry(geom, lod, vertices):
    indices = []
    
    if geom.type == "LineString":
        geom_type = "MultiLineString"
        
        # Sort of hacky: if it's a LineString you expect a single-level array.
        indices = process_geometry(geom, vertices)
    elif geom.type == "MultiLineString":
        geom_type = "MultiLineString"
        
        indices = process_geometry(geom, vertices)
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
        
        indices = [process_geometry(geom.boundary, vertices)]
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": indices
    }

def create_cityobject(feature, vertices):
    return {
        "type": "Road",
        "attributes": {
            "osm_id": feature['osmid'],
            "highway": feature['highway'],
        },
        "geometry": [
            create_geometry(feature['geometry'], "0.1", vertices)
        ]
    }

Now, let's run this against all intersected road segments:

In [ ]:
vertices = []

objects = {i: create_cityobject(f, vertices) for i, f in roads_dissolved.iterrows()}

for i, f in bgt_all_roads.iterrows():
    if f['gml_id'] in objects:
        objects[f['gml_id']]['geometry'].append(create_geometry(f['geometry'], "2", vertices))

Finally, let's export everything as CityJSON:

In [ ]:
import json
import io

output = {
  "type": "CityJSON",
  "version": "1.0",
  "CityObjects": objects,
  "vertices": vertices
}

with open('output/breda.json', 'w') as file:
    json.dump(output, file)